<a href="https://colab.research.google.com/github/neeti098/SharpCareer/blob/main/API_development.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install flask-ngrok
!pip install flask
!pip install surprise


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357248 sha256=f7d85d73c7c406e920779726641327938a2432d409dc27e9b5d3ab67e56ab928
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from flask import Flask, request, jsonify
import pickle

# Load the data
df = pd.read_csv('/content/Purchase Data.csv')
df['Review Rating'] = (df['Review Rating'] - 2.5) / 2.5
df['Purchase Amount (USD)'] = (df['Purchase Amount (USD)'] - 20) / 80

# Model Preparation
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[['Customer ID', 'Item Purchased', 'Purchase Amount (USD)']], reader)
trainset, testset = train_test_split(data, test_size=0.25, random_state=42)

algo = SVD()
algo.fit(trainset)

# Save the model
with open('recommendation_model.pkl', 'wb') as f:
    pickle.dump(algo, f)

# Prepare content-based recommendations
product_features = df[['Category', 'Size', 'Color', 'Season', 'Review Rating']].copy()
product_features = pd.get_dummies(product_features)
cosine_sim = cosine_similarity(product_features)

def get_content_based_recommendations(item_purchased, cosine_sim=cosine_sim):
    idx = df[df['Item Purchased'] == item_purchased].index[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    item_indices = [i[0] for i in sim_scores]
    return df['Item Purchased'].iloc[item_indices].tolist()

def hybrid_recommendations(customer_id, item_purchased, algo, cosine_sim, weight_cf=0.5, weight_cb=0.5):
    cf_preds = algo.predict(customer_id, item_purchased).est
    cb_preds = get_content_based_recommendations(item_purchased, cosine_sim)
    cb_preds_mean = df[df['Item Purchased'].isin(cb_preds)]['Purchase Amount (USD)'].mean()
    hybrid_preds = weight_cf * cf_preds + weight_cb * cb_preds_mean
    return cf_preds, cb_preds_mean, hybrid_preds

def get_top_n_recommendations(user_id, algo, cosine_sim, top_n=10):
    hybrid_ratings = {}
    for item in df['Item Purchased'].unique():
        cf_pred, cb_pred, hybrid_pred = hybrid_recommendations(user_id, item, algo, cosine_sim)
        hybrid_ratings[item] = hybrid_pred
    sorted_items = sorted(hybrid_ratings.items(), key=lambda x: x[1], reverse=True)
    top_n_items = [item[0] for item in sorted_items[:top_n]]
    return top_n_items

# Create Flask app
app = Flask(__name__)

@app.route('/')
def home():
    return "Welcome to the Recommendation API!"

@app.route('/recommend', methods=['GET'])
def recommend():
    user_id = int(request.args.get('user_id'))
    item_purchased = request.args.get('item_purchased')

    with open('recommendation_model.pkl', 'rb') as f:
        algo = pickle.load(f)

    recommendations = get_top_n_recommendations(user_id, algo, cosine_sim)

    return jsonify(recommendations)

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
